Milestone2


In [99]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

# kagglehub.login()


Dataset available at: https://www.kaggle.com/datasets/tonygordonjr/football-match-statistics-and-more/data?fbclid=IwZXh0bgNhZW0CMTEAAR27HPyyqC9nUioeOqJzVzznFoIH3SNY0xcdZ_4dDxEpDOoRnZgcaXdTTRE_aem_0dX5wF1-vk6WmlQWZMnQ2g

In [100]:
# !kaggle datasets download -d tonygordonjr/football-match-statistics-and-more
# !unzip /content/football-match-statistics-and-more.zip -d ./

In [101]:
teams_clean_col = ['fixture_id', 'team_id', 'team_name', 'fouls','yellow_cards', 'red_cards']
stats_clean_col = ['fixture_id', 'fixture_date', 'fixture_referee', 'league_id', 'league_name', 'teams_home_id', 'teams_home_name', 'teams_away_id', 'teams_away_name']
players_clean_col = ['fixture_id', 'team_id', 'team_name', 'player_id', 'player_name', 'fouls_committed', 'yellow_cards', 'red_cards', 'game_minutes']

fix_teams_clean = pd.read_csv("fixture_stats_teams_clean.csv",usecols=teams_clean_col)
fix_clean = pd.read_csv("fixture_stats_clean.csv", usecols=stats_clean_col)
fix_players_clean = pd.read_csv("fixture_stats_players_clean.csv",usecols=players_clean_col)

In [102]:
fix_players_clean['total_cards'] = fix_players_clean.apply(lambda x: x['yellow_cards'] + x['red_cards'] * 2 if x['yellow_cards'] != 2 else 3, axis=1)
fix_players_clean.sort_values('red_cards', ascending=False).head()

,fixture_id,team_id,team_name,player_id,player_name,game_minutes,fouls_committed,yellow_cards,red_cards,total_cards
98098,12291,36,Fulham,49923,Aboubakar Kamara,63.0,5,0,1,2
1000312,1149546,628,Sparta Praha,295027,Matěj Ryneš,80.0,2,2,1,3
290364,194669,163,Borussia Monchengladbach,1464,Granit Xhaka,86.0,6,2,1,3
132123,571820,80,Lyon,651,Marcelo,90.0,1,0,1,2
963004,592363,48,West Ham,1243,Tomáš Souček,90.0,0,0,1,2


In [103]:
fix_players_clean.drop(['yellow_cards', 'red_cards'], axis=1, inplace=True)

In [104]:
fix_total_cards = fix_players_clean.groupby('fixture_id')['total_cards'].sum().reset_index()

In [105]:
fix_players_clean.drop(fix_players_clean[fix_players_clean["game_minutes"].isna()].index, inplace=True)

In [106]:
fix_players_clean["game_minutes"].isna().sum()

np.int64(0)

In [107]:
fix_players_clean_og = fix_players_clean.merge(fix_clean[['fixture_id', 'fixture_date']], on='fixture_id', how='left')

In [108]:
def calculate_avg_cards_per_90(df):
    # Sort by fixture_id to ensure chronological order
    df = df.sort_values(by=['player_name', 'fixture_date'])

    # Create new columns for storing cumulative stats
    df['cumulative_cards'] = 0
    df['cumulative_fouls'] = 0
    df['cumulative_minutes'] = 0
    df['avg_cards_per_90'] = 0.0
    df['avg_fouls_per_90'] = 0.0

    # Group by player_name and iterate to compute averages from past matches
    grouped = df.groupby('player_name')

    for player_name, player_df in grouped:
        cumulative_cards = 0
        cumulative_fouls = 0
        cumulative_minutes = 0

        for idx, row in player_df.iterrows():
            # Compute averages from past matches
            if cumulative_minutes > 0:
                avg_cards = (cumulative_cards / cumulative_minutes) * 90
                avg_fouls = (cumulative_fouls / cumulative_minutes) * 90
            else:
                avg_cards = 0
                avg_fouls = 0

            # Store calculated averages
            df.at[idx, 'avg_cards_per_90'] = avg_cards
            df.at[idx, 'avg_fouls_per_90'] = avg_fouls

            # Update cumulative stats
            cumulative_cards += row['total_cards']
            cumulative_fouls += row['fouls_committed']
            cumulative_minutes += row['game_minutes']

    return df

In [109]:
fix_players_clean = calculate_avg_cards_per_90(fix_players_clean_og)

In [110]:
fix_players_clean_final = fix_players_clean.groupby('fixture_id').apply(lambda x: x.nlargest(22, 'game_minutes')).reset_index(drop=True)

<ipython-input-110-b11d4ccd1a40>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fix_players_clean_final = fix_players_clean.groupby('fixture_id').apply(lambda x: x.nlargest(22, 'game_minutes')).reset_index(drop=True)


In [111]:
top_leagues = ["La Liga", "Serie A", "Bundesliga", "Premier League"]
fix_clean = fix_clean[fix_clean["league_name"].isin(top_leagues)]

In [112]:
teams_to_keep = list(set(list(fix_clean.teams_home_name.unique())+list(fix_clean.teams_away_name.unique())))

In [113]:
print(len(fix_players_clean_final))
fix_players_clean_final = fix_players_clean_final[fix_players_clean_final['team_name'].isin(teams_to_keep)]
print(len(fix_players_clean_final))

713628
396033


In [114]:
fix_players_clean_final['player_num'] = fix_players_clean_final.groupby('fixture_id').cumcount() + 1

In [115]:
pivoted_fix_players_clean_final = fix_players_clean_final.pivot_table(
    index='fixture_id',
    columns='player_num',
    values=['avg_cards_per_90', 'avg_fouls_per_90']
)

In [116]:
pivoted_fix_players_clean_final.columns = [
    f'player_{num}_avg_cards_per_90' if stat == 'avg_cards_per_90'
    else f'player_{num}_avg_fouls_per_90'
    for stat, num in pivoted_fix_players_clean_final.columns
]

In [117]:
pivoted_fix_players_clean_final.reset_index(inplace=True)

In [118]:
pivoted_fix_players_clean_final

,fixture_id,player_1_avg_cards_per_90,player_2_avg_cards_per_90,player_3_avg_cards_per_90,player_4_avg_cards_per_90,player_5_avg_cards_per_90,player_6_avg_cards_per_90,player_7_avg_cards_per_90,player_8_avg_cards_per_90,player_9_avg_cards_per_90,player_10_avg_cards_per_90,player_11_avg_cards_per_90,player_12_avg_cards_per_90,player_13_avg_cards_per_90,player_14_avg_cards_per_90,player_15_avg_cards_per_90,player_16_avg_cards_per_90,player_17_avg_cards_per_90,player_18_avg_cards_per_90,player_19_avg_cards_per_90,player_20_avg_cards_per_90,player_21_avg_cards_per_90,player_22_avg_cards_per_90,player_1_avg_fouls_per_90,player_2_avg_fouls_per_90,player_3_avg_fouls_per_90,player_4_avg_fouls_per_90,player_5_avg_fouls_per_90,player_6_avg_fouls_per_90,player_7_avg_fouls_per_90,player_8_avg_fouls_per_90,player_9_avg_fouls_per_90,player_10_avg_fouls_per_90,player_11_avg_fouls_per_90,player_12_avg_fouls_per_90,player_13_avg_fouls_per_90,player_14_avg_fouls_per_90,player_15_avg_fouls_per_90,player_16_avg_fouls_per_90,player_17_avg_fouls_per_90,player_18_avg_fouls_per_90,player_19_avg_fouls_per_90,player_20_avg_fouls_per_90,player_21_avg_fouls_per_90,player_22_avg_fouls_per_90
0,65,0.143312,0.424318,0.188453,0.015748,0.078295,0.360000,0.208360,0.144552,0.062730,0.111340,0.161871,0.221311,0.152975,0.080190,0.102041,0.225362,0.225836,0.383591,0.544721,0.111268,0.211931,0.171662,1.227110,1.518610,0.753814,0.000000,0.567638,1.213333,0.983923,0.644922,0.008961,1.595876,0.930755,1.235656,1.223796,0.694981,0.909091,1.738508,2.174721,1.726159,1.331540,0.953727,1.448195,1.029973
1,66,0.141956,0.016251,0.168724,0.179402,0.125244,0.007116,0.162560,0.173793,0.307439,0.050647,0.000000,0.112251,0.127861,0.169743,0.316648,0.167574,0.181850,0.105464,0.205597,0.202507,0.075847,0.219097,0.798502,0.520043,0.688955,1.016611,1.018647,0.007116,0.557347,0.637241,0.967864,1.164885,0.000000,1.908259,1.534333,0.581975,1.367344,1.095675,0.964596,1.502856,1.559109,1.243973,1.274229,1.655396
2,67,0.127407,0.033046,0.054711,0.028880,0.313534,0.160085,0.144491,0.063416,0.098576,0.319863,0.178277,0.022222,0.077187,0.106200,0.222092,0.143908,0.210628,0.000000,0.164303,0.392328,0.031480,0.166282,0.662514,0.033046,0.724924,0.279174,1.473611,1.120598,1.381698,1.775648,1.456736,1.220219,0.876527,0.000000,0.540309,0.975113,1.173913,0.508475,1.251377,1.724891,0.973178,1.333915,0.409234,1.829099
3,68,0.143312,0.289505,0.179334,0.211634,0.066667,0.203466,0.213371,0.143019,0.306122,0.273141,0.054370,0.170481,0.031791,0.102772,0.033333,0.200793,0.141121,0.090310,0.255936,0.131538,0.226540,0.000000,0.286624,1.917973,0.922289,1.381188,0.066667,1.722683,0.668563,0.817253,1.107443,1.980273,0.362465,1.008682,0.747086,0.551230,0.000000,1.561725,0.576245,0.451548,2.201052,0.883182,1.123641,0.000000
4,69,0.136326,0.225268,0.000000,0.161983,0.104408,0.292290,0.208012,0.315021,0.264317,0.095420,0.178887,0.174300,0.338983,0.171429,0.051282,0.207900,0.323477,0.156596,0.105634,0.156160,0.132331,0.082619,0.892316,1.330155,0.000000,0.635470,0.819204,1.042079,1.109399,1.438137,1.392748,0.019084,1.083260,1.403118,1.728814,1.714286,0.769231,1.039501,2.161412,1.024992,0.786385,0.659341,1.161575,0.578335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20024,1299226,0.138614,0.104881,0.242644,0.294510,0.195886,0.185028,0.339623,0.210400,0.184995,0.325733,0.120443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.772277,0.006169,1.030360,1.079869,0.822723,1.404039,1.912290,1.192264,0.961973,2.182410,0.806602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20025,1299227,0.000000,0.000000,0.199262,0.412214,0.352041,0.317460,0.176256,0.294011,0.551095,0.187857,0.101115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.026616,0.730627,1.723803,1.392857,1.836735,1.119462,0.777495,1.739012,0.826569,1.097825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20026,1299231,0.203812,0.1177

In [119]:
# TODO select top 22 players for each match by game minutes and add their avg_cards_per_90 and avg_fouls_per_90 as separate columns

In [120]:
# import pandas as pd


# # Clean all data before 2020. jan. 1.
# fix_clean["fixture_date"] = pd.to_datetime(fix_clean["fixture_date"]).dt.tz_localize(None)
# start_date = pd.Timestamp("2020-01-01")
# fix_clean = fix_clean[fix_clean["fixture_date"] >= start_date]


# print(fix_clean.head())



In [121]:

count3 = fix_clean.shape[0]

# Keep only the 4 main leagues
top_leagues = ["La Liga", "Serie A", "Bundesliga", "Premier League"]
fix_clean = fix_clean[fix_clean["league_name"].isin(top_leagues)]

count4 = fix_clean.shape[0]
print(f"Number of rows before filtering: {count3}")
print(f"Number of rows after filtering: {count4}")

Number of rows before filtering: 19755
Number of rows after filtering: 19755


In [122]:
# count1 = fix_teams_clean.shape[0]

# # Drop all unneccessary teams' datas
# team_ids_to_keep = fix_players_clean["team_id"].unique()
# fix_teams_clean = fix_teams_clean[fix_teams_clean["team_id"].isin(team_ids_to_keep)]
# count2 = fix_teams_clean.shape[0]

# print(f"Number of rows before filtering: {count1}")
# print(f"Number of rows after filtering: {count2}")

In [123]:
print(fix_players_clean.columns)
print(fix_teams_clean.columns)
print(fix_clean.columns)

Index(['fixture_id', 'team_id', 'team_name', 'player_id', 'player_name',
       'game_minutes', 'fouls_committed', 'total_cards', 'fixture_date',
       'cumulative_cards', 'cumulative_fouls', 'cumulative_minutes',
       'avg_cards_per_90', 'avg_fouls_per_90'],
      dtype='object')
Index(['fixture_id', 'team_id', 'team_name', 'fouls', 'yellow_cards',
       'red_cards'],
      dtype='object')
Index(['fixture_id', 'fixture_date', 'fixture_referee', 'league_id',
       'league_name', 'teams_home_id', 'teams_home_name', 'teams_away_id',
       'teams_away_name'],
      dtype='object')


In [124]:

merged = fix_clean.merge(fix_teams_clean, on="fixture_id", how="inner")
merged = merged[merged["team_id"] == merged["teams_home_id"]]
merged = merged.rename(columns={"fouls": "home_team_fouls", "yellow_cards": "home_team_yellow_cards", "red_cards": "home_team_red_cards"})
merged = merged.drop(columns=["team_id", "team_name"])

merged = merged.merge(fix_teams_clean, on="fixture_id", how="inner")
merged = merged[merged["team_id"] == merged["teams_away_id"]]
merged = merged.rename(columns={"fouls": "away_team_fouls", "yellow_cards": "away_team_yellow_cards", "red_cards": "away_team_red_cards"})
merged = merged.drop(columns=["team_id", "team_name"])



In [125]:
pd.set_option('display.max_columns', 100)
merged[merged['fixture_id'] == 1208526]

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,home_team_fouls,home_team_yellow_cards,home_team_red_cards,away_team_fouls,away_team_yellow_cards,away_team_red_cards
13335,1208526,2024-08-31 19:30:00 UTC,Mario Melero,140,La Liga,537,Leganes,798,Mallorca,11,1,0,17,3,0


In [126]:
# Needed in the final table:
#fixture referee, league_name, teams_home_name, teams_away_name, avg_home_team_fouls, avg_home_team_yellow_cards, avg_home_team_red_cards, avg_away_team_fouls, avg_away_team_yellow_cards, avg_away_team_red_cards, [avg_total_cards_player_{1-22}, avg_total_fouls_player_{1_22},] total_cards(yellow_cards+red_cards*2)


In [127]:
# TODO 1 - DONE
# Meg kéne csinálni az átlagokat home és away csapatra is szabálytalanságokból, sárga és piros lapokból
# LEHET EGYSZERŰBB EGYBŐL TODO 2-VEL PRÓBÁLKOZNI!

# merged['avg_home_team_fouls'] = merged.groupby('teams_home_name')['home_team_fouls'].transform('mean')
# merged['avg_home_team_yellow_cards'] = merged.groupby('teams_home_name')['home_team_yellow_cards'].transform('mean')
# merged['avg_home_team_red_cards'] = merged.groupby('teams_home_name')['home_team_red_cards'].transform('mean')
# merged['avg_away_team_fouls'] = merged.groupby('teams_away_name')['away_team_fouls'].transform('mean')
# merged['avg_away_team_yellow_cards'] = merged.groupby('teams_away_name')['away_team_yellow_cards'].transform('mean')
# merged['avg_away_team_red_cards'] = merged.groupby('teams_away_name')['away_team_red_cards'].transform('mean')

In [128]:
merged

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,home_team_fouls,home_team_yellow_cards,home_team_red_cards,away_team_fouls,away_team_yellow_cards,away_team_red_cards
0,566789,2020-07-06 18:30:00 UTC,"Felix Brych, Germany",78,Bundesliga,180,FC Heidenheim,162,Werder Bremen,16,2,0,7,2,0
3,566788,2020-07-02 18:30:00 UTC,F. Zwayer,78,Bundesliga,162,Werder Bremen,180,FC Heidenheim,7,4,1,16,2,0
4,9822,2017-10-01 10:00:00 UTC,"Alfonso Alvarez Izquierdo, Spain",140,La Liga,548,Real Sociedad,543,Real Betis,9,3,0,17,3,0
6,203592,2016-02-27 21:05:00 UTC,"Alberto Undiano, Spain",140,La Liga,548,Real Sociedad,535,Malaga,0,4,0,0,5,0
8,203530,2016-01-16 21:05:00 UTC,"Juan Martinez Munuera, Spain",140,La Liga,548,Real Sociedad,544,Deportivo La Coruna,0,1,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25664,878055,2022-10-29 12:00:00 UTC,Carlos del Cerro,140,La Liga,723,Almeria,538,Celta Vigo,10,4,0,7,0,1
25666,878215,2023-04-09 16:30:00 UTC,José Sánchez,140,La Liga,723,Almeria,532,Valencia,11,3,0,11,2,0
25668,877967,2022-08-27 20:00:00 UTC,Jesús Gil,140,La Liga,723,Almeria,536,Sevilla,18,5,0,11,8,0
25670,878075,2022-11-09 18:00:00 UTC,Alejandro Muñiz,140,La Liga,723,Almeria,546,Getafe,17,8,1,8,4,0


In [129]:
# TODO 2 - DONE
# Az átlagokat úgy kéne megcsinálni hogy csak az azon meccs előtti infóknak legyen az átlaga

avgs = ['home_team_fouls', 'home_team_yellow_cards', 'home_team_red_cards', 'away_team_fouls', 'away_team_yellow_cards', 'away_team_red_cards']
merged = merged.sort_values(by=["teams_home_id", "fixture_date"])
for avg in avgs:
    merged[f"avg_{avg}"] = merged.groupby("teams_home_id")[avg] \
    .expanding().mean().shift(1).reset_index(level=0, drop=True).fillna(0)

merged["avg_home_team_cards"] = merged["avg_home_team_yellow_cards"] + merged["avg_home_team_red_cards"] * 2
merged["avg_away_team_cards"] = merged["avg_away_team_yellow_cards"] + merged["avg_away_team_red_cards"] * 2

merged["total_cards"] = merged["home_team_yellow_cards"] + merged["away_team_yellow_cards"] + merged["home_team_red_cards"] * 2 + merged["away_team_red_cards"] * 2

final_cols = ["fixture_id", "fixture_referee", "league_name", "teams_home_name", "teams_away_name", "avg_home_team_fouls", "avg_away_team_fouls","avg_home_team_cards","avg_away_team_cards", "total_cards"]
merged = merged[final_cols]
merged



,fixture_id,fixture_referee,league_name,teams_home_name,teams_away_name,avg_home_team_fouls,avg_away_team_fouls,avg_home_team_cards,avg_away_team_cards,total_cards
16065,192297,"Jonathan Moss, England",Premier League,Manchester United,Tottenham,0.000000,0.000000,0.000000,0.000000,5
913,192317,"Craig Pawson, England",Premier League,Manchester United,Newcastle,0.000000,0.000000,2.000000,3.000000,4
16040,192362,"Mike Jones, England",Premier League,Manchester United,Sunderland,0.000000,0.000000,2.000000,2.500000,3
917,192394,"Mark Clattenburg, England",Premier League,Manchester United,Manchester City,0.000000,0.000000,1.333333,2.666667,4
15982,192412,"Mike Dean, England",Premier League,Manchester United,West Brom,0.000000,0.000000,1.500000,2.500000,6
...,...,...,...,...,...,...,...,...,...,...
2538,1223662,F. La Penna,Serie A,Monza,AS Roma,11.756098,13.658537,2.317073,2.731707,4
2546,1223680,A. Rapuano,Serie A,Monza,Venezia,11.857143,13.642857,2.333333,2.690476,8
9655,1223700,E. Feliciani,Serie A,Monza,AC Milan,11.976744,13.697674,2.418605,2.674419,3
9659,1223712,A. Colombo,Serie A,Monza,Lazio,12.113636,13.727273,2.409091,2.636364,6


In [130]:
merged_final = merged.merge(pivoted_fix_players_clean_final, on="fixture_id", how="left")
merged_final

,fixture_id,fixture_referee,league_name,teams_home_name,teams_away_name,avg_home_team_fouls,avg_away_team_fouls,avg_home_team_cards,avg_away_team_cards,total_cards,player_1_avg_cards_per_90,player_2_avg_cards_per_90,player_3_avg_cards_per_90,player_4_avg_cards_per_90,player_5_avg_cards_per_90,player_6_avg_cards_per_90,player_7_avg_cards_per_90,player_8_avg_cards_per_90,player_9_avg_cards_per_90,player_10_avg_cards_per_90,player_11_avg_cards_per_90,player_12_avg_cards_per_90,player_13_avg_cards_per_90,player_14_avg_cards_per_90,player_15_avg_cards_per_90,player_16_avg_cards_per_90,player_17_avg_cards_per_90,player_18_avg_cards_per_90,player_19_avg_cards_per_90,player_20_avg_cards_per_90,player_21_avg_cards_per_90,player_22_avg_cards_per_90,player_1_avg_fouls_per_90,player_2_avg_fouls_per_90,player_3_avg_fouls_per_90,player_4_avg_fouls_per_90,player_5_avg_fouls_per_90,player_6_avg_fouls_per_90,player_7_avg_fouls_per_90,player_8_avg_fouls_per_90,player_9_avg_fouls_per_90,player_10_avg_fouls_per_90,player_11_avg_fouls_per_90,player_12_avg_fouls_per_90,player_13_avg_fouls_per_90,player_14_avg_fouls_per_90,player_15_avg_fouls_per_90,player_16_avg_fouls_per_90,player_17_avg_fouls_per_90,player_18_avg_fouls_per_90,player_19_avg_fouls_per_90,player_20_avg_fouls_per_90,player_21_avg_fouls_per_90,player_22_avg_fouls_per_90
0,192297,"Jonathan Moss, England",Premier League,Manchester United,Tottenham,0.000000,0.000000,0.000000,0.000000,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,192317,"Craig Pawson, England",Premier League,Manchester United,Newcastle,0.000000,0.000000,2.000000,3.000000,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.826087,0.000000,0.526316,0.000000,0.000000,1.500000,1.000000,0.000000,0.500000,0.000000,1.052632,1.000000,0.500000,1.500000,0.500000,0.600000,3.000000,0.000000,0.000000,0.000000,1.139241,0.500000,7.826087,0.000000,1.052632,0.523256,1.500000,3.000000
2,192362,"Mike Jones, England",Premier League,Manchester United,Sunderland,0.000000,0.000000,2.000000,2.500000,3,0.000000,0.000000,0.535714,0.166667,0.166667,0.000000,1.232877,0.342205,0.000000,0.173077,0.332103,0.380550,0.209302,0.166667,0.000000,0.000000,0.829493,0.176471,0.000000,0.387931,0.000000,0.000000,1.649215,1.267606,2.500000,1.166667,0.000000,0.000000,2.465753,1.711027,0.666667,0.519231,0.996310,1.331924,2.511628,0.833333,0.800000,0.800000,1.244240,1.058824,0.333333,1.551724,0.389610,0.000000
3,192394,"Mark Clattenburg, England",Premier League,Manchester United,Manchester City,0.000000,0.000000,1.333333,2.666667,4,0.097403,0.480000,0.108173,0.090909,0.090909,0.000000,0.339943,0.539461,0.000000,0.118421,0.312500,0.227848,0.324324,0.000000,0.342205,0.100000,0.190678,0.000000,0.210280,0.427215,0.108827,0.000000,0.876623,1.920000,1.514423,0.454545,1.181818,0.000000,2.209632,1.348651,0.000000,1.302632,1.875000,2.620253,1.297297,1.261682,1.197719,0.900000,0.762712,0.791789,1.471963,1.708861,0.435308,0.637544
4,192412,"Mike Dean, England",Premier League,Manchester United,West Brom,0.000000,0.000000,1.500000,2.500000,6,0.083488,0.312139,0.000000,0.142857,0.413318,0.090909,0.000000,0.090909,0.000000,0.356671,0.000000,0.000000,0.171920,0.000000,0.000000,0.111111,0.302691,0.076923,0.322581,0.144695,0.000000,0.386266,1.586271,1.560694,0.000000,1.142857,1.756602,0.454545,0.719281,0.454545,0.000000,1.426684,1.445783,0.642857,0.515759,0.909091,1.000000,1.000000,0.908072,0.923077,1.612903,0.434084,0.691421,1.545064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.